<a href="https://colab.research.google.com/github/newmantic/SABR/blob/main/SABR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

def sabr_implied_volatility(F, K, T, alpha, beta, rho, nu):
    """
    Calculate the implied volatility using the SABR model.

    Parameters:
    - F: Forward rate
    - K: Strike rate
    - T: Time to maturity
    - alpha: SABR alpha parameter (initial volatility)
    - beta: SABR beta parameter (elasticity)
    - rho: SABR rho parameter (correlation)
    - nu: SABR nu parameter (volatility of volatility)

    Returns:
    - sigma_impl: Implied volatility
    """
    if F == K:
        # At-the-money case
        FK = F * K
        logFK = 0
    else:
        FK = F * K
        logFK = np.log(F / K)

    z = nu / alpha * (FK ** ((1 - beta) / 2)) * logFK
    x_z = np.log((np.sqrt(1 - 2 * rho * z + z ** 2) + z - rho) / (1 - rho))

    A = alpha / ((FK) ** ((1 - beta) / 2))
    B = 1 + (((1 - beta) ** 2 / 24) * (logFK ** 2) + (rho * beta * nu / 4) * logFK + ((2 - 3 * rho ** 2) / 24) * (nu ** 2)) * T

    sigma_impl = A * z / x_z * B

    return sigma_impl

def sabr_swaption_price(F, K, T, alpha, beta, rho, nu, notional, option_type='payer'):
    """
    Calculate the swaption price using the SABR model.

    Parameters:
    - F: Forward rate
    - K: Strike rate
    - T: Time to maturity
    - alpha: SABR alpha parameter (initial volatility)
    - beta: SABR beta parameter (elasticity)
    - rho: SABR rho parameter (correlation)
    - nu: SABR nu parameter (volatility of volatility)
    - notional: Notional amount of the swaption
    - option_type: 'payer' for payer swaption, 'receiver' for receiver swaption

    Returns:
    - swaption_price: Price of the swaption
    """
    sigma_impl = sabr_implied_volatility(F, K, T, alpha, beta, rho, nu)
    d1 = (np.log(F / K) + 0.5 * sigma_impl ** 2 * T) / (sigma_impl * np.sqrt(T))
    d2 = d1 - sigma_impl * np.sqrt(T)

    if option_type == 'payer':
        swaption_price = notional * (F * norm.cdf(d1) - K * norm.cdf(d2))
    elif option_type == 'receiver':
        swaption_price = notional * (K * norm.cdf(-d2) - F * norm.cdf(-d1))
    else:
        raise ValueError("Invalid option type. Choose 'payer' or 'receiver'.")

    return swaption_price

In [2]:
from scipy.stats import norm

# Parameters
F = 0.02           # Forward rate (2%)
K = 0.025          # Strike rate (2.5%)
T = 1.0            # Time to maturity (1 year)
alpha = 0.02       # Initial volatility (2%)
beta = 0.5         # Elasticity parameter
rho = -0.3         # Correlation
nu = 0.4           # Volatility of volatility (40%)
notional = 1000000 # Notional amount

# Calculate implied volatility
implied_vol = sabr_implied_volatility(F, K, T, alpha, beta, rho, nu)
print(f"SABR Implied Volatility: {implied_vol:.4f}")

# Calculate payer swaption price
payer_swaption_price = sabr_swaption_price(F, K, T, alpha, beta, rho, nu, notional, option_type='payer')
print(f"Payer Swaption Price: {payer_swaption_price:.2f}")

SABR Implied Volatility: 0.1322
Payer Swaption Price: 55.53
